# Optimal project allocation

## Simple Score-Based Allocation

In [9]:
import pandas as pd
import numpy as np
from pulp import LpMaximize, LpProblem, LpVariable, lpSum

def standardize_project_name(project):
    """清理项目名字前后空格"""
    if isinstance(project, str):
        return project.strip()
    return None

def read_excel_and_prepare_scores(filename):
    # 读取表单
    df = pd.read_excel(filename)

    # 学生唯一标识：Student ID
    students = df["Student ID"].astype(str).tolist()
    
    # 项目选择列
    choice_columns = [
        'Project (First Choice)',
        'Project (Second Choice)',
        'Project (Third Choice)',
        'Project (Forth Choice)',   # 注意今年还是 "Forth"
        'Project (Fifth Choice)'
    ]
    
    # 收集所有唯一项目
    projects = set()
    for col in choice_columns:
        df[col] = df[col].apply(standardize_project_name)
        projects.update(df[col].dropna().unique())
    projects = list(projects)
    
    # 构建得分矩阵
    scores = pd.DataFrame(0, index=students, columns=projects)
    for idx, row in df.iterrows():
        sid = str(row["Student ID"])
        for rank, col in enumerate(choice_columns):
            project = row[col]
            if project:
                scores.at[sid, project] = 5 - rank  # 第一志愿5分，第二志愿4分 ...
    
    return df, students, projects, scores

def optimize_allocation(students, projects, scores, project_capacity, project_lower_bound):
    # 定义优化问题
    prob = LpProblem("Project_Allocation", LpMaximize)
    
    # 决策变量：x[s, p] = 1 表示学生s分配到项目p
    x = LpVariable.dicts("assign", [(s, p) for s in students for p in projects], cat='Binary')
    
    # 目标函数：最大化总得分
    prob += lpSum([scores.at[s, p] * x[(s, p)] for s in students for p in projects])
    
    # 每个学生只能分配一个项目
    for s in students:
        prob += lpSum([x[(s, p)] for p in projects]) <= 1
    
    # 每个项目容量约束
    for p in projects:
        prob += lpSum([x[(s, p)] for s in students]) <= project_capacity.get(p, 6)
        prob += lpSum([x[(s, p)] for s in students]) >= project_lower_bound.get(p, 5)
    
    prob.solve()
    
    # 生成分配结果
    allocation = {}
    for s in students:
        for p in projects:
            if x[(s, p)].varValue == 1:
                allocation[s] = p
                break
        else:
            allocation[s] = None
    return allocation

def main():
    input_filename = '2025-2026 MGT 555 - Project Selection Form (Responses).xlsx'
    output_filename = 'score_based_allocation_results_2025_2026.xlsx'
    
    # 读取数据
    df, students, projects, scores = read_excel_and_prepare_scores(input_filename)
    
    # 👉 打印所有唯一项目，供确认
    print("\n📌 本年度唯一项目列表：")
    for i, p in enumerate(projects, 1):
        print(f"{i}. {p}")
    
    # 默认容量：上限6，下限5
    project_capacity = {p: 6 for p in projects}
    project_lower_bound = {p: 5 for p in projects}
    
    # 优化分配
    allocation = optimize_allocation(students, projects, scores, project_capacity, project_lower_bound)
    
    # 构建输出结果
    results = []
    choice_columns = [
        'Project (First Choice)',
        'Project (Second Choice)',
        'Project (Third Choice)',
        'Project (Forth Choice)',
        'Project (Fifth Choice)'
    ]
    
    for _, row in df.iterrows():
        sid = str(row["Student ID"])
        assigned_project = allocation.get(sid)
        preference_rank = None
        if assigned_project:
            for rank, col in enumerate(choice_columns):
                if row[col] == assigned_project:
                    preference_rank = rank + 1
                    break
        results.append({
            'Student ID': sid,  # ✅ 学号
            'First Name': row.get('First Name', ''),
            'Last Name': row.get('Last Name', ''),
            'Email': row.get('Email address', ''),
            'Assigned Project': assigned_project if assigned_project else 'Not Assigned',
            'Preference Rank': preference_rank,
            'Educational Background': row.get('Educational Background', ''),
            'Major (Program)': row.get('Major (Program)', ''),
            'MTE Minor': row.get('MTE Minor (for any other minor use other)', '')
        })
    
    results_df = pd.DataFrame(results)

    # 按 Last Name + First Name 排序
    results_df = results_df.sort_values(by=['Last Name', 'First Name']).reset_index(drop=True)

    # 导出结果
    results_df.to_excel(output_filename, index=False)
    print(f"\n✅ Assignment results have been saved to '{output_filename}'.")

if __name__ == '__main__':
    main()



📌 本年度唯一项目列表：
1. Wearin-AI-Enhanced Light Beacon for Firefighter Rescue
2. Humard-Self-learning robot for part feeding
3. Decathlon-AI-Powered Shelf Layout and Customer Flow Optimization
4. Deep-Scouting-Low-Cost 3D Football Match Reconstruction
5. Wearin-AI-Powered Firefighter Warning Assistant
6. Humard-Self-learning robot for autonomous part handling
7. Decathlon-Autonomous Robotic Restocking Assistant

✅ Assignment results have been saved to 'score_based_allocation_results_2025_2026.xlsx'.


## 📌 Two-Stage Optimization Algorithm (Team Allocation with Top-3 Guarantee)

### 🎯 Objective
To allocate students to projects such that as many students as possible receive one of their **top-3 preferences**, while also maximizing the overall satisfaction score.

---

### 📝 Algorithm Design

#### Stage 1: Maximize Top-3 Coverage
- **Decision variables**
  - \(x_{s,p} \in \{0,1\}\): whether student \(s\) is assigned to project \(p\).  
  - \(y_s \in \{0,1\}\): whether student \(s\) is assigned to one of their top-3 choices.  

- **Constraints**
  1. Each student is assigned to exactly one project:  
     $$
     \sum_{p} x_{s,p} = 1 \quad \forall s
     $$
  2. Each project must satisfy capacity bounds:  
     $$
     L_p \leq \sum_{s} x_{s,p} \leq U_p \quad \forall p
     $$
  3. Linking variable \(y_s\) to top-3 preferences:  
     $$
     y_s \leq \sum_{p \in \text{Top3}(s)} x_{s,p} \quad \forall s
     $$

- **Objective function**  
  Maximize the number of students assigned within their top-3 choices:  
  $$
  \max \sum_s y_s
  $$

---

#### Stage 2: Maximize Satisfaction under Top-3 Guarantee
- **Carry-over constraint from Stage 1**  
  Ensure at least \(\text{max\_top3}\) students are in top-3 choices:  
  $$
  \sum_s y_s \geq \text{max\_top3}
  $$

- **Satisfaction scores**
  - 1st choice: 5 points  
  - 2nd choice: 4 points  
  - 3rd choice: 3 points  
  - 4th choice: 2 points  
  - 5th choice: 1 point  

- **Objective function**  
  Maximize the total satisfaction score:  
  $$
  \max \sum_{s,p} \text{score}(s,p) \cdot x_{s,p}
  $$

---

### 🌟 Advantages
1. **Fairness**: maximizes the number of students receiving top-3 choices.  
2. **Satisfaction**: further improves overall allocation quality after fairness.  
3. **Flexibility**: project capacity bounds can be tuned.  
4. **Transparency**: results include *Preference Rank* and *Top3 (Yes/No)* labels.

---

### 📊 Output
The final results table contains:
- Student ID  
- First Name, Last Name  
- Email  
- Assigned Project  
- Preference Rank (which choice)  
- Top3 (Yes/No)  
- Additional fields: Educational Background, Major, Minor, etc.  



In [4]:
import pandas as pd
from pulp import LpMaximize, LpProblem, LpVariable, lpSum

def standardize_project_name(project):
    if isinstance(project, str):
        return project.strip()
    return None

def read_excel_and_prepare_scores(filename):
    df = pd.read_excel(filename)
    students = df["Student ID"].astype(str).tolist()
    choice_columns = [
        'Project (First Choice)',
        'Project (Second Choice)',
        'Project (Third Choice)',
        'Project (Forth Choice)',
        'Project (Fifth Choice)'
    ]
    projects = set()
    for col in choice_columns:
        df[col] = df[col].apply(standardize_project_name)
        projects.update(df[col].dropna().unique())
    projects = list(projects)

    scores = pd.DataFrame(0, index=students, columns=projects)
    for idx, row in df.iterrows():
        sid = str(row["Student ID"])
        for rank, col in enumerate(choice_columns):
            project = row[col]
            if project:
                scores.at[sid, project] = 5 - rank
    return df, students, projects, scores, choice_columns

def optimize_allocation_two_stage(df, students, projects, scores, choice_columns,
                                  project_capacity, project_lower_bound):
    x = LpVariable.dicts("assign", [(s, p) for s in students for p in projects], cat='Binary')
    y = LpVariable.dicts("in_top3", students, cat='Binary')

    # -------- 阶段1: 最大化前三志愿覆盖人数 --------
    prob1 = LpProblem("Stage1_MaxTop3", LpMaximize)
    for s in students:
        prob1 += lpSum([x[(s, p)] for p in projects]) == 1
    for p in projects:
        prob1 += lpSum([x[(s, p)] for s in students]) <= project_capacity.get(p, 6)
        prob1 += lpSum([x[(s, p)] for s in students]) >= project_lower_bound.get(p, 5)
    for s in students:
        row = df[df["Student ID"] == int(s)].iloc[0]
        top3 = [row[col] for col in choice_columns[:3]]
        # top2  = [row[col] for col in choice_columns[:2]]
        prob1 += y[s] <= lpSum([x[(s, p)] for p in top3 if p in projects])
        # prob1 += y[s] <= lpSum([x[(s, p)] for p in top2 if p in projects])
    prob1 += lpSum([y[s] for s in students])
    prob1.solve()
    max_top3 = sum(y[s].varValue for s in students)
    # max_top2 = sum(y[s].varValue for s in students)
    print(f"✅ 阶段1完成：最多 {max_top3} 名学生分到前三志愿")
    # print(f"✅ 阶段1完成：最多 {max_top2} 名学生分到前二志愿")

    # -------- 阶段2: 在保持覆盖人数的前提下最大化总得分 --------
    prob2 = LpProblem("Stage2_MaxScore", LpMaximize)
    for s in students:
        prob2 += lpSum([x[(s, p)] for p in projects]) == 1
    for p in projects:
        prob2 += lpSum([x[(s, p)] for s in students]) <= project_capacity.get(p, 6)
        prob2 += lpSum([x[(s, p)] for s in students]) >= project_lower_bound.get(p, 5)
    for s in students:
        row = df[df["Student ID"] == int(s)].iloc[0]
        top3 = [row[col] for col in choice_columns[:3]]
        # top2  = [row[col] for col in choice_columns[:2]]
        prob2 += y[s] <= lpSum([x[(s, p)] for p in top3 if p in projects])
        # prob2 += y[s] <= lpSum([x[(s, p)] for p in top2 if p in projects])
    prob2 += lpSum([y[s] for s in students]) >= max_top3
    # prob2 += lpSum([y[s] for s in students]) >= max_top2
    prob2 += lpSum([scores.at[s, p] * x[(s, p)] for s in students for p in projects])
    prob2.solve()

    allocation = {}
    for s in students:
        for p in projects:
            if x[(s, p)].varValue == 1:
                allocation[s] = p
                break
        else:
            allocation[s] = None
    return allocation, max_top3
    # return allocation, max_top2

def main():
    input_filename = '2025-2026 MGT 555 - Project Selection Form (Responses).xlsx'
    output_filename = 'score_based_allocation_results_2025_2026.xlsx'

    df, students, projects, scores, choice_columns = read_excel_and_prepare_scores(input_filename)
    project_capacity = {p: 6 for p in projects}
    project_lower_bound = {p: 5 for p in projects}

    allocation, max_top3 = optimize_allocation_two_stage(
        df, students, projects, scores, choice_columns, project_capacity, project_lower_bound
    )

    results = []
    for _, row in df.iterrows():
        sid = str(row["Student ID"])
        assigned_project = allocation.get(sid)
        preference_rank = None
        if assigned_project:
            for rank, col in enumerate(choice_columns):
                if row[col] == assigned_project:
                    preference_rank = rank + 1
                    break
        results.append({
            'Student ID': sid,
            'First Name': row.get('First Name', ''),
            'Last Name': row.get('Last Name', ''),
            'Email': row.get('Email address', ''),
            'Assigned Project': assigned_project if assigned_project else 'Not Assigned',
            'Preference Rank': preference_rank,
            'Top3': "Yes" if preference_rank and preference_rank <= 3 else "No",
            'Educational Background': row.get('Educational Background', ''),
            'Major (Program)': row.get('Major (Program)', ''),
            'MTE Minor': row.get('MTE Minor (for any other minor use other)', '')
        })

    results_df = pd.DataFrame(results)
    results_df = results_df.sort_values(by=['Last Name', 'First Name']).reset_index(drop=True)
    results_df.to_excel(output_filename, index=False)

    print(f"\n✅ 最终结果已保存到 '{output_filename}'")
    print(f"📊 共有 {max_top3} 名学生分到前三志愿")

if __name__ == "__main__":
    main()


✅ 阶段1完成：最多 40.0 名学生分到前三志愿

✅ 最终结果已保存到 'score_based_allocation_results_2025_2026.xlsx'
📊 共有 40.0 名学生分到前三志愿
